In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

import numpy as np
import pandas as pd
import datetime as datetime
import matplotlib as mp
import matplotlib.pyplot  as plt
import seaborn as sns


Read the file as pandas dataframe

In [ ]:
taxi_train_df =  pd.read_csv('../input/train.csv', nrows = 1_000_000, parse_dates=["pickup_datetime"])

In [ ]:
taxi_train_df.head()

Lets check for null values

In [ ]:
taxi_train_df.isnull().sum()

Lets remove the records which contains null values in dropoff_longitude and dropoff_latitude 

In [ ]:
taxi_train_df=taxi_train_df.loc[taxi_train_df.dropoff_longitude.notnull() & taxi_train_df.dropoff_latitude.notnull(),:]
taxi_train_df.isnull().sum()

Now to find out outliers in this data , lets calculate the distance between pickup and dropoff points. Lets approximate this distance to eucladian distance in kilomenters and calculate price per kilometer. Than we will inspect this distance later to remove outliers

In [ ]:
import calendar
from math import radians,sin, cos, sqrt, atan2,acos


def get_distance(lat1,lon1,lat2,lon2):
    dist=0
    if((lat1!=lat2) | (lon1!=lon2)):
        slat = radians(lat1)
        slon = radians(lon1)
        elat = radians(lat2)
        elon = radians(lon2)
        dist = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
    #distance = 6371.01 * acos(sin(lat1)*sin(lat2) + cos(lat1)*cos(lat2)*cos(lon1 - lon2))    
    return dist


# Lets find out the outliers to find these lets first calculate the  euladian distance between laattitude and logitude
# divide total  fare by the distance

taxi_train_df["euc_dis"]=taxi_train_df.apply(lambda df:get_distance(df['pickup_latitude'],df['pickup_longitude'],df['dropoff_latitude'],df['dropoff_longitude']), axis=1)


Lets Extract week day and hour of day from pickup_datetime and check describe() on the data frame

In [ ]:
taxi_train_df["week_day"]=taxi_train_df["pickup_datetime"].dt.day_name()
taxi_train_df["hour"]=taxi_train_df["pickup_datetime"].dt.hour

taxi_train_df.describe()

We can see some euc_dis as 0 lets take a count and remove these records

In [ ]:
taxi_train_df[taxi_train_df.euc_dis==0].count()

In [ ]:
taxi_train_df[(taxi_train_df.pickup_longitude==taxi_train_df.dropoff_longitude) & (taxi_train_df.pickup_latitude==taxi_train_df.dropoff_latitude)].count()

This confirms that for tese records latitude and longitude of pickup and dropoff points are equal, Lets check fare of some of these records

In [ ]:
taxi_train_df[(taxi_train_df.pickup_longitude==taxi_train_df.dropoff_longitude) & (taxi_train_df.pickup_latitude==taxi_train_df.dropoff_latitude)].head()['fare_amount']

These may be the case of cancelled trip lets encode the same in a new variable 'is_completed' as '1' for cancelled and '0' for completed

In [ ]:
taxi_train_df.loc[((taxi_train_df.pickup_longitude==taxi_train_df.dropoff_longitude) & (taxi_train_df.pickup_latitude==taxi_train_df.dropoff_latitude)) | (taxi_train_df.euc_dis==0),'is_completed']=0
taxi_train_df.loc[np.invert((taxi_train_df.pickup_longitude==taxi_train_df.dropoff_longitude) & (taxi_train_df.pickup_latitude==taxi_train_df.dropoff_latitude)) & (taxi_train_df.euc_dis!=0),'is_completed']=1
taxi_train_df.describe()

Lets calculate price per km, by dividing fare_amount with euc_dis , i.e. price_per_km=fare_amount/euc_dis

In [ ]:
taxi_train_df.loc[taxi_train_df.is_completed==1 & taxi_train_df.euc_dis.notnull(),'price_per_km']=taxi_train_df['fare_amount']/taxi_train_df['euc_dis']
taxi_train_df.loc[taxi_train_df.is_completed==0 & taxi_train_df.euc_dis.isnull(),'price_per_km']=taxi_train_df['fare_amount']
taxi_train_df.describe()

Lets check if there are any more nulls or NaNs in the data frame

In [ ]:
taxi_train_df.isnull().sum()

No Null or NaNs observed.

Lets check the outliers in the dataframe. To do this we will use price_per_km variable, and we will calculate the deciles, quartiles and also use Boxplot and calculate Inter Quartile Range and remove the Outliers. The next few steps describes the same

In [ ]:
taxi_train_df['price_per_km'].quantile(np.arange(0,1.1,.1))

In [ ]:
taxi_train_df.loc[taxi_train_df.price_per_km<0,:]

In [ ]:
taxi_train_df.loc[taxi_train_df.price_per_km>1.246708e+06,:]

In [ ]:
# Lets remove this record as this is a outlier
taxi_train_df=taxi_train_df.loc[taxi_train_df.price_per_km<1.246708e+06,:]

In [ ]:
# Lets remove the records with fare_amount as negetive
taxi_train_df=taxi_train_df[taxi_train_df.fare_amount>0]

In [ ]:
# Lets remove the records where euc_dis is 0 and fare is greter than 0
taxi_train_df=taxi_train_df.loc[np.invert((taxi_train_df.euc_dis==0) & (taxi_train_df.fare_amount>0)),:]

In [ ]:
taxi_train_df['price_per_km'].describe()

In [ ]:
taxi_train_df[taxi_train_df.price_per_km>895345]

In [ ]:
# We observe that lots of distance are less verry close to 0 which has lead to increase in fare amount, lets remove these records
taxi_train_df=taxi_train_df[(taxi_train_df.euc_dis>.1)]

In [ ]:
taxi_train_df['price_per_km'].quantile(np.arange(0,1.1,.1))

In [ ]:
# Lets remove all records with price_per_km greater than 2 digits
from math import log10
taxi_train_df=taxi_train_df[(np.log10(taxi_train_df.price_per_km) + 1).astype(int)<=2]

taxi_train_df[np.log10(taxi_train_df.price_per_km).astype(int)>2]

In [ ]:
taxi_train_df['price_per_km'].quantile(np.arange(0,1.1,.1))


Lets do a box plot and check for outliers in fare_amount

In [ ]:
#visualise_numeric(taxi_train_df,'price_per_km','fare_amount')
sns.boxplot(y='price_per_km',data=taxi_train_df)

In [ ]:
sns.distplot(taxi_train_df['price_per_km'],bins=50)

In [ ]:
# Lets remove outliers using Inter quartile ranges
Q1 = taxi_train_df['price_per_km'].quantile(0.25)
Q3 = taxi_train_df['price_per_km'].quantile(0.75)
IQR = Q3 - Q1
print(Q1)
print(Q3)
print(IQR)
print(Q1 - 1.5 * IQR)
print((Q3 + 1.5 * IQR))
taxi_train_df_out_rem=taxi_train_df[((taxi_train_df.price_per_km >= (Q1 - 1.5 * IQR)) & (taxi_train_df.price_per_km <= (Q3 + 1.5 * IQR)))]

In [ ]:
taxi_train_df_out_rem['price_per_km'].quantile(np.arange(0,1.1,.1))


In [ ]:
sns.distplot(taxi_train_df_out_rem['price_per_km'],bins=50)

In [ ]:
#visualise_numeric(taxi_train_df,'price_per_km','fare_amount')
sns.boxplot(y='price_per_km',data=taxi_train_df_out_rem)

In [ ]:
taxi_train_df_out_rem['price_per_km'].quantile(np.arange(0,1.1,.1))


 Now the data seems free of outliers
 
 Lets check how much % data we removed as outliers and due to data issue out of 100000 

In [ ]:
((1000000-914098)/1000000)*100

which comes out nearly 8.6 %

Lets check the outliers in passenger_count

In [ ]:
sns.distplot(taxi_train_df_out_rem['passenger_count'],bins=50)

No Outliers Observed

In [ ]:
# Lets plot the points on xy axis and check if we can apply simple liear regression on it
sns.scatterplot(x="pickup_longitude", y="fare_amount", data=taxi_train_df_out_rem)

In [ ]:
# Lets plot the points on xy axis and check if we can apply simple liear regression on it
sns.scatterplot(x="pickup_latitude", y="fare_amount", data=taxi_train_df_out_rem)

In [ ]:
# Lets plot the points on xy axis and check if we can apply simple liear regression on it
sns.scatterplot(x="dropoff_longitude", y="fare_amount", data=taxi_train_df_out_rem)

In [ ]:
sns.scatterplot(x="dropoff_latitude", y="fare_amount", data=taxi_train_df_out_rem)

In [ ]:
sns.scatterplot(x="week_day", y="fare_amount", data=taxi_train_df_out_rem)

In [ ]:
sns.barplot(x="hour", y="fare_amount", data=taxi_train_df_out_rem, estimator=sum)

In [ ]:
sns.barplot(x="hour", y="fare_amount", data=taxi_train_df_out_rem, estimator=np.mean)

lets biin hour the same into :-
 3:00 to 07:00  as EARLY_MORNING
 8:00 to 11:00 as MORNING
 12:00 to 17:00 as AFTERNOON
 18:00 to 22:00 as EVENING
 23:00 to 02:00 as LATE_EVENING

In [ ]:
def get_time_labels(x):
    if x>=3 and x<=7:
        return 'EARLY_MORNING'
    elif x>=8 and x<=11:
        return 'MORNING'
    elif x>=12 and x<=17:
        return 'AFTERNOON'
    elif x>=18 and x<=22:
        return 'EVENING'
    elif x==23 or x==0 or (x>=1 and x<=2):
        return 'LATE_EVENING'
    else:
        raise ValueError('Please input values betweenb 0 to 23')
    

    
taxi_train_df_out_rem.loc[:,'DAY_TIME']=taxi_train_df_out_rem.apply(lambda x:get_time_labels(x.hour),axis=1)

In [ ]:
taxi_train_df_out_rem.head()

Collecting the variables for final modelling

In [ ]:
taxi_train_df_noenc=taxi_train_df_out_rem.loc[:,['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','week_day','DAY_TIME','fare_amount']]

# lets do one hot encoding
taxi_train_df_enc=pd.get_dummies(taxi_train_df_noenc, drop_first=True)

In [ ]:
taxi_train_df_enc.head()

Creating training and Test Sets

In [ ]:
from sklearn.model_selection import train_test_split

X=taxi_train_df_enc.drop('fare_amount',axis=1)

y=taxi_train_df_enc[['fare_amount']]

X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=.7,random_state=100)

# Lets check if means are comparable
print(y_train.mean(),y_test.mean())

Implementing XGBoost regression algorithm with default parameters to solve this problem

In [ ]:

import xgboost as xgb

from xgboost import XGBRegressor

model=XGBRegressor()
model.fit(X_train,y_train)


Lets do Prediction on test set and check for R-Square as a measure of accuracy

In [ ]:
y_pred=model.predict(X_test)

# Lets evaluate the results

from sklearn.metrics import mean_squared_error,r2_score

mse=mean_squared_error(y_test,y_pred)
r_square=r2_score(y_test,y_pred)
print("mse:",mse)
print("r_square:",r_square)

We get R-square or .8041. Now lets try to improve the same using 3- Fold cross validation (Since 'cv' is not mentioned in GridSearchCV  it is 3 by default)

In [ ]:
# parameter grid
parameter_grid={"learning_rate":[0.2,0.6,0.9],"subsample":[0.3,0.6,0.9]}

# specify a model
xgb_model=XGBRegressor(max_depth=2,n_estimators=200)

# setup a GridSearch
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

model_cv=GridSearchCV(estimator=xgb_model,param_grid=parameter_grid,cv=3,scoring="r2")

model_cv.fit(X_train,y_train)


In [ ]:
cv_results=pd.DataFrame(model_cv.cv_results_)
cv_results

Lets create a model with best fit hyper parameters results we got from Cross validation and check for R-Square

In [ ]:
xgb_model_final=XGBRegressor(max_depth=2,n_estimators=200,learning_rate=0.6,subsample=0.9)
xgb_model_final.fit(X_train,y_train)

y_pred=xgb_model_final.predict(X_test)
mse=mean_squared_error(y_test,y_pred)
r_square=r2_score(y_test,y_pred)
print("mse:",mse)
print("r_square:",r_square)

The R-Square we got is 0.84 which is greater than 0.8 what we got from default model

Lets predict the fare_price for given test data

In [ ]:
taxi_test_df =  pd.read_csv('../input/test.csv', parse_dates=["pickup_datetime"])
taxi_test_df=taxi_test_df.loc[taxi_test_df.dropoff_longitude.notnull() & taxi_test_df.dropoff_latitude.notnull(),:]
taxi_test_df["week_day"]=taxi_test_df["pickup_datetime"].dt.day_name()
taxi_test_df["hour"]=taxi_test_df["pickup_datetime"].dt.hour
taxi_test_df.loc[:,'DAY_TIME']=taxi_test_df.apply(lambda x:get_time_labels(x.hour),axis=1)
taxi_test_df_noenc=taxi_test_df.loc[:,['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','week_day','DAY_TIME']]
taxi_test_df_key=taxi_test_df['key']
taxi_test_df_enc=pd.get_dummies(taxi_test_df_noenc, drop_first=True)
y_pred=xgb_model_final.predict(taxi_test_df_enc)

result_df=pd.DataFrame({"key":taxi_test_df_key,"fare_amount":y_pred})

In [ ]:
result_df.head()